In [1]:

from kinesis_stream import KinesisStream


stream2 = KinesisStream('office2-outsight-objects', create_if_not_found=False)
stream3 = KinesisStream('office3-outsight-objects', create_if_not_found=False)



In [54]:
records_gen2 = stream2.get_records_iter()
records_gen3 = stream3.get_records_iter()

In [37]:
def record_to_frame(record):
    lines = s.decode('utf-8').splitlines()
    frame = lines[0].split(',')

    frame_dict = {
        'frame_Count': int(frame[1]),
        'time_s': frame[2],
        'formatted_time': frame[3],
        'number_of_objects': int(frame[4]),
        'zone_bindings_len': int(frame[5]),
        'objects': [],
        }


    objects = lines[1:1+frame_dict['number_of_objects']]
    obj = dict()
    for object in objects:
        object = object.split(',')
        obj = {
            'frame_count': int(object[1]),
            'obj_id': int(object[2]),
            'object_class': object[3],
            'pos_x': float(object[4]),
            'pos_y': float(object[5]),
            'pos_z': float(object[6]),
            'dim_x': float(object[7]),
            'dim_y': float(object[8]),
            'dim_z': float(object[9]),
            'speed_mph': float(object[10]),
            'bearing_degrees': float(object[11]),
        }

        frame_dict['objects'].append(obj)
    return frame_dict

In [ ]:
for record2, record3 in zip(records_gen2, records_gen3):
    if len(records_gen2) > 0:
        frame2 = record_to_frame(record2[0]['Data'])
    if len(records_gen3) > 0:
        frame3 = record_to_frame(record3[0]['Data'])
    
    

In [55]:
import json
ops = []
counter = 2
for record in records_gen:
    if len(record) > 0: 
        ops.append(record_to_frame(record[0]['Data']))
    if len(ops) == 1000:
        filename = f'ops_{counter}.json'
        with open(filename, 'w') as outfile:
            json.dump(ops, outfile)
        counter = counter+1
        ops = []

KeyboardInterrupt: 

In [3]:
import json

filename = f'ops_{1}.json'
with open(filename, 'w') as outfile:
  json.dump(ops, outfile)



NameError: name 'ops' is not defined

In [24]:
s = b'FRAMEv1,5088,1716388531.16,2024-05-22 14:35:31.157321,1,0\nOBJv1,5088,15542,UNKNOWN,11.54,-0.60,0.33,0.02,0.12,0.65,3,-85\nZONEv1,5088,NONE,NONE,NONE\n'


In [13]:
import json
with open('ops_2.json', 'r') as outfile:
    op = json.load(outfile)

In [14]:
objects = op[0]['objects']

In [2]:
from data_classes import Box

In [23]:
from pyquaternion import Quaternion
import numpy as np
boxes = []
for obj in objects:
    yaw_angle = obj['bearing_degrees'] * np.pi / 180
    yaw_quaternion = Quaternion(axis=[0, 0, 1], angle=yaw_angle)
    center = [obj['pos_x'], obj['pos_y'], obj['pos_z']]
    size = [obj['dim_x'], obj['dim_y'], obj['dim_z']]
    box = Box(center, size, yaw_quaternion, velocity = [obj['speed_mph'], 0, 0]) # Convert this to 3D velocity
    boxes.append(box)

In [24]:
box

label: nan, score: nan, xyz: [11.54, -0.60, 0.33], wlh: [0.02, 0.12, 0.65], rot axis: [-0.00, -0.00, -1.00], ang(degrees): 85.00, ang(rad): 1.48, vel: 3.00, 0.00, 0.00, name: None, token: None

In [51]:
def string2array(data_string):

    lines = data_string.replace("[", "").replace("]", "")

    lines = lines.strip().splitlines()
    data = []

    for line in lines:
        cleaned_line = line.strip().split()

        data.append([float(x) for x in cleaned_line])

    array = np.array(data)
    return array


In [52]:
calibration_string = """[[ 0.2314198  -0.         -0.97285398  0.        ]
 [ 0.          1.         -0.          0.        ]
 [ 0.97285398  0.          0.2314198  -2.01845491]
 [ 0.          0.          0.          1.        ]]"""
calibration_matrix = string2array(calibration_string)

In [ ]:
box.